# 📈 Volatility Surface Estimation & SVI Smile Fitting
This notebook analyzes **TSLA** options, computing implied volatilities using Newton-Raphson with JAX auto-differentiation, constructing the implied volatility surface, and fitting a smooth SVI model.

**Tools used:** `Python`, `JAX`, `SciPy`, `Matplotlib`, `yfinance`

## 🛠️ Imports and Setup

In [ ]:
import jax.numpy as jnp
from jax import grad
from jax.scipy.stats import norm as jnorm
from datetime import datetime
import pandas as pd
import yfinance as yf
from scipy.interpolate import griddata
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


ticker = yf.Ticker("TSLA")
expiration = ticker.options
print("Available Options:\n", expiration)
expiry = expiration[5]
opt_chain = ticker.option_chain(expiry)
calls = opt_chain.calls
puts = opt_chain.puts

spot_price = ticker.info['regularMarketPrice']
print("Spot Price:", spot_price)

expiry_date = datetime.strptime(expiry, '%Y-%m-%d')
today = datetime.today()
T = (expiry_date - today).days/365
print("Time to Expiry (in years):", T)

strike_prices_calls = calls['strike']
strike_prices_puts = puts['strike']

def blackscholes(S, K, T, r, sigma, q=0, otype="call"):
    d1 = (jnp.log(S/K) + (r-q+0.5*sigma**2)* T)/((sigma)*jnp.sqrt(T))
    d2 = d1 - sigma * jnp.sqrt(T)

    if otype == "call":
        call = (S * jnp.exp(-q*T)*jnorm.cdf(d1, 0, 1 ) - K*jnp.exp(-r*T)*
                jnorm.cdf(d2, 0, 1))
        return call
    else:
        put = K*jnp.exp(-r*T)*jnorm.cdf(-d2, 0, 1) - S * jnp.exp(-q*T)* jnorm.cdf(-d1, 0, 1 )
        return put



In [ ]:
#returns theoretical price - market price
def loss(S, K, T, r, sigma, price, q=0, otype="call"):
    return blackscholes(S, K, T, r, sigma, q, otype) - price

loss_grad = grad(loss, argnums=4)



In [ ]:
#returns the only prices which are appropriate and correct
def is_price_valid(S, K, price, otype):
    if otype == "call":
        intrinsic = max(0, S-K)
    else:
        intrinsic = max(0, K-S)
    return price > intrinsic


def solving_iv(S, K, T, r, price, sigma_guess=0.2, q=0, otype="call", n_iter=50, epsilon=0.001, check=True):
    sigma = sigma_guess
    sigma = jnp.clip(sigma, 0.01, 4)
    for i in range(n_iter):
        loss_val = loss(S, K, T, r, sigma, price, q, otype)
        if check:
            print(f"Iteration {i}: σ = {sigma:.4f}, Loss = {loss_val:.6f}")
        if abs(loss_val) < epsilon:
            return sigma
        grad_val = loss_grad(S, K, T, r, sigma, price, q, otype)
        if jnp.isnan(loss_val) or grad_val == 0:
            return jnp.nan
        sigma -= loss_val / grad_val
        sigma = jnp.clip(sigma, 0.01, 4.0)
    return jnp.nan

r=0.05

all_data = []

for expiry in expiration:
    expiry_date = datetime.strptime(expiry, "%Y-%m-%d")
    T = (expiry_date - today).days / 365
    if T <= 0:  # Expired
        continue

    opt_chain = ticker.option_chain(expiry)

    for idx, row in opt_chain.calls.iterrows():
        K = row['strike']
        if jnp.isnan(row['bid']) or jnp.isnan(row['ask']) or row['bid'] == 0 or row['ask'] == 0:
            continue

        price = (row['bid'] + row['ask']) / 2
        spread = row['ask'] - row['bid']
        if spread / price > 0.3:
            continue

        if not is_price_valid(spot_price, K, price, "call"):
            continue

        iv = solving_iv(spot_price, K, T, r, price, otype="call", check=False)
        if iv < 0.01 or iv > 5 or jnp.isnan(iv):
            continue

        log_m = jnp.log(spot_price/K)
        all_data.append([T, K, log_m, iv, row['bid'], row['ask'], row['volume'], spread])

iv_df = pd.DataFrame(
    all_data,
    columns=["T", "Strike", "log_m", "IV", "Bid", "Ask", "Volume", "Spread"]
)


In [ ]:
# Sanity and quality filters
iv_df = iv_df.dropna()
iv_df = iv_df[iv_df["Ask"] > iv_df["Bid"]]
iv_df = iv_df[iv_df["Bid"] > 0]
iv_df = iv_df[iv_df["IV"] > 0.01]
iv_df = iv_df[iv_df["IV"] < 5]



In [ ]:
# Log-moneyness range (filtering for clean SVI fitting)
iv_df = iv_df[(iv_df["log_m"] > -1.0) & (iv_df["log_m"] < 1.0)]



In [ ]:
# Maturity bounds
iv_df = iv_df[(iv_df["T"] > 7/365) & (iv_df["T"] < 180/365)]



In [ ]:
# Drop duplicate strikes per expiry (optional, but safe)
iv_df = iv_df.drop_duplicates(subset=["T", "Strike"])

print("Number of rows after filtering:", len(iv_df))
print("Unique expiries left:", iv_df["T"].nunique())
print("T values:", iv_df["T"].unique())
print(f"Final cleaned options: {len(iv_df)}")

T_grid = np.linspace(iv_df["T"].min(), iv_df["T"].max(), 30)
log_moneyness_grid = np.linspace(iv_df["log_m"].min(), iv_df["log_m"].max(), 30)
T_mesh, M_mesh = np.meshgrid(T_grid, log_moneyness_grid)

IV_grid = griddata(
    points=(iv_df["T"], iv_df["log_m"]),
    values=iv_df["IV"],
    xi=(T_mesh, M_mesh),
    method='linear'
)

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection="3d")

ax.plot_surface(M_mesh, T_mesh, IV_grid, cmap="viridis")
ax.set_xlabel("Log_Moneyness (S/K)")
ax.set_ylabel("Time to Maturity (Years)")
ax.set_zlabel("Implied Volatility")
plt.show()

atm_slice = iv_df[iv_df['T'].between(0.1, 0.2)]  # e.g., ≈1-2 months to expiry
plt.figure(figsize=(8, 4))
plt.scatter(atm_slice["Strike"], atm_slice["IV"], alpha=0.6)
plt.title("Volatility Smile (T ≈ 1–2 months)")
plt.xlabel("Strike Price K")
plt.ylabel("Implied Volatility")
plt.grid(True)
plt.show()



In [ ]:
#fitting a volatility smile using a stochastic volatility inspired model
target_T = 30/365
iv_df["T_diff"] = (iv_df["T"]-target_T).abs()


In [ ]:
# Filter for a *range* of expiries, e.g., 20 to 60 days
iv_df = iv_df[(iv_df["T"] >= 20/365) & (iv_df["T"] <= 60/365)]
best_slice = iv_df.loc[iv_df.groupby("T_diff")["T_diff"].idxmin()].drop(columns = "T_diff")
T_chosen = best_slice["T"].iloc[0]
svi_slice = iv_df[iv_df["T"] == T_chosen].copy()

x = svi_slice["log_m"].values
iv = svi_slice["IV"].values
T = svi_slice["T"].iloc[0]
w = (iv ** 2) * T

def svi_raw(x, a, b, rho, m, sigma):
    return a+b*(rho*(x-m)+jnp.sqrt((x-m)**2+sigma**2))

initial_guess = [0.01, 0.1, -0.3, 0.0, 0.1]  # [a, b, rho, m, sigma]
bounds = ([0, 0, -1, -jnp.inf, 0], [jnp.inf, jnp.inf, 1, jnp.inf, jnp.inf])

params, _ = curve_fit(svi_raw, x, w, p0=initial_guess, bounds=bounds)

x_range = np.linspace(x.min(), x.max(), 100)
w_fit = svi_raw(x_range, *params)
iv_fit = np.sqrt(w_fit / T)

plt.figure(figsize=(10, 6))
plt.scatter(x, iv, label="Market IV", color="blue")
plt.plot(x_range, iv_fit, label="SVI Fit", color="red")
plt.xlabel("Log Moneyness")
plt.ylabel("Implied Volatility")
plt.title(f"SVI Fit - T = {round(T * 365)} days")
plt.grid(True)
plt.legend()
plt.show()

## ⚙️ Implied Volatility Estimation
Use Newton-Raphson with JAX autodiff to numerically invert Black-Scholes.

## 🌐 Volatility Surface Plot
Interpolate IVs to build a 3D surface across time and moneyness.

## 😊 Volatility Smile Snapshot
Extract and plot a 1–2 month expiry smile.

## 🔧 SVI Calibration
Fit the SVI model to the extracted volatility smile.